In [3]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
training_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())
print(training_data.train_data)

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

C:\Users\Loona\.conda\envs\tf310\lib\site-packages\torchvision\datasets\mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for images, labels in train_dataloader:
    print(f"Shape of Images: {images.shape}")
    print(f"Shape of labels: {labels.shape}")
    break

In [ ]:
weird_shape = np.ndarray([4, 1, 2, 3])
print(f"Shape of weird: {weird_shape.shape}")
print(weird_shape)

Set the default computing device.

In [ ]:
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Device: {device}")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, training_data):
        training_data = self.flatten(training_data)
        logits = self.linear_relu_stack(training_data)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

How the training work
1. Use the model to do a prediction (at first it will be really bad because the ai is basically randomized)
2. Take the result and calculate the difference using the loss function
3. backpropagate the loss function result and optimize using the optimizer

In [ ]:
def train(dataloader: torch.utils.data.DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss,
          optimizer: torch.optim.SGD):
    size = len(dataloader.dataset)
    model.train()
    for batch, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        
        # Compute prediction error
        prediction = model(images)
        loss = loss_fn(prediction, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(images) # 64 Images
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
                
train(train_dataloader, model, loss_fn, optimizer)

In [ ]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
print(logits)
prediction_probabilities = nn.Softmax(dim=1)(logits)
print(prediction_probabilities)
label_prediction = prediction_probabilities.argmax(1)
print(f"Label Prediction: {label_prediction}")

In [ ]:
def test(dataloader: torch.utils.data.DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            prediction = model(images)
            test_loss += loss_fn(prediction, labels).item()
            correct += (prediction.argmax(1) == labels).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

In [ ]:
# Save Model
torch.save(model.state_dict(), "./model.pth")
print("Saved PyTorch model state to model.pth")

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
    
with torch.no_grad():
    for i in range(len(test_data)):
        image, label = test_data[i][0].to(device), test_data[i][1]
        prediction = model(image)
        predicted, actual = classes[prediction[0].argmax(0)], classes[label]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
x = torch.tensor(1, dtype=torch.float)
x = torch.rand_like(x, dtype=torch.float)
x.item()

In [22]:
# Define the matrix
matrix = [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]

# Get the diagonal from bottom-left to top-right
diagonal = [row[i] for i, row in enumerate(matrix[::-1])]

print(matrix[0][::-1])

[3, 2, 1]
